In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import variational_autoencoder as vae
import weapon_data as weapons

print("Tensor Flow version {}".format(tf.__version__))

# Test Utils

In [ ]:
def print_decoded_tensors_as_dict(weapon_data, array_of_tensors):
    genDict = {}
    
    for tensor in array_of_tensors:
        decoded, _ = weapon_data.decode_processed_tensor(tensor)
        
        for key, value in decoded.items():
            if key not in genDict:
                genDict[key] = []
            genDict[key].append(value)
    
    for key, value in genDict.items():
            print(key, "=", value)
            
            
def get_weapon_data():
    return weapons.get_data()

# Initial VAE Training
Initializes all network hyperparameters and shows training debug messages of the training epoch and cost. Trains and saves the trained model in the specified folder.

In [ ]:
network_architecture = \
    dict(n_input=0, #set it in with scope
         n_hidden_1=26,
         n_hidden_2=12,
         n_z=2)  

learning_rate = 0.01
optimizer =  tf.train.RMSPropOptimizer(learning_rate)
transfer_fct = tf.nn.elu
num_epochs = 70
batch_size = 4
epoch_debug_step = 1
saved_model_folder = "trained_vae/"
saved_model_full_path = saved_model_folder + "model.ckpt"
    
with tf.Session() as sess:
    train_data, test_data = get_weapon_data()
    network_architecture['n_input'] = train_data.num_features
  
    network = vae.get_new_trained(sess, train_data, network_architecture, optimizer, 
                              transfer_fct, batch_size, num_epochs, epoch_debug_step,
                                  trained_model_save_path=saved_model_folder)


# Encode and Decode Testing #1 
Tests the encoding and decoding functionality and outputs the inputted and generated values. This case uses the same size as the training batch_size.

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, saved_model_full_path)
        
    train_data, test_data = get_weapon_data()
    
    samples = test_data.next_batch(batch_size)
    x_reconstructed = network.encode_and_decode(samples, True)
    print_decoded_tensors_as_dict(test_data, np.concatenate((samples,x_reconstructed), axis=0))

# Encode and Decode Testing #2
Tests the encoding and decoding functionality and outputs the inputted and generated values. This case does not use the same size as the training batch_size.

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, saved_model_full_path)

    train_data, test_data = get_weapon_data()
    
    samples = test_data.next_batch(1)
    x_reconstructed_mean = network.encode_and_decode(samples, False)
    print_decoded_tensors_as_dict(test_data, np.concatenate((samples,[x_reconstructed_mean]), axis=0))

# Latent Space Visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_z_distribution(vae_model, title, z_mean=True):
    all_z = np.zeros((batch_size,network_architecture['n_z']))
        
    train_data, test_data = get_weapon_data()
    total_batch = int(train_data.num_examples / batch_size) 

    # Loop over all batches
    for i in range(total_batch):
        batch = train_data.next_batch(batch_size)
        z_dist = vae_model.calculate_z(batch)
        if z_mean:
            z_dist = vae_model.calculate_z_mean(batch)
        all_z = np.vstack((all_z, z_dist))

    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.scatter(all_z[:,0], all_z[:,1])
    plt.xlim(-3,3)
    plt.ylim(-3,3)
    plt.title(title)
    plt.subplot(1,2,2)
    plt.hist2d(all_z[:,0], all_z[:,1], (50, 50), cmap=plt.cm.jet)
    plt.colorbar()
    plt.title(title)
    
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    show_z_distribution(network, "Untrained Latent Space", z_mean=True)
    network = vae.restore(network, "trained_vae/model.ckpt")
    show_z_distribution(network, "Trained Latent Space - Z Mean", z_mean=True)    
    show_z_distribution(network, "Trained Latent Space - Z", z_mean=False)

# Random Input Decoding Test #1
This tests the decoding from latent space functionality with random input. This case does not use the same size as the training batch_size.

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, saved_model_full_path)
    
    generated = []

    random_val = np.random.normal(size=(1,network_architecture["n_z"]))
    x_test = network.decode_from_latent_space(random_val, False)
    #[generated.append(x) for x in x_test]
    generated.append(x_test)  
    
    train_data, test_data = get_weapon_data()
    print_decoded_tensors_as_dict(train_data, generated)

# Random Input Decoding Test #2
This tests the decoding from latent space functionality with random input. This case uses the same size as the training batch_size.

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, saved_model_full_path)
    
    generated = []

    random_val = np.random.normal(size=(batch_size,network_architecture["n_z"]))
    x_test = network.decode_from_latent_space(random_val, True)
    [generated.append(x) for x in x_test]
    
    train_data, test_data = get_weapon_data()
    print_decoded_tensors_as_dict(train_data, generated)